### This Notebook is for exploring the IFG network
- plot bad IFGs
- perp baseline plots
- coherence matrix

In [ ]:
#Import statements
import os
import numpy as np
import matplotlib.pyplot as plt
import rioxarray as rioxr
import glob
import pandas as pd

from mintpy.utils import readfile, utils as ut, plot as pp
from mintpy.cli import view, tsview, plot_network, plot_transection
from mintpy.view import prep_slice, plot_slice

In [ ]:
#Set up directories
track = 'T28'

work_dir = os.path.expanduser('/data/eme66/PROCESSDIR/EAR/{}/mintpy/'.format(track))
ifg_dir = os.path.expanduser('/data/eme66/PROCESSDIR/EAR/{}/merged/interferograms/'.format(track))

##### Plot the interferogram network

In [ ]:
# Make sure you're in the right directory
os.chdir(work_dir)
print('Now in work directory: ', work_dir)

plot_network.main('inputs/ifgramStack.h5 -t smallbaselineApp.cfg --figsize 8 4'.split())

In [ ]:
# Make sure you're in the right directory
os.chdir(work_dir)
print('Now in work directory: ', work_dir)

# Print all IFGs with an average spatial coherence < threshold

cohth = 0.4 #coherence threshold - choose based on what makes sense

ifg_coh = pd.read_csv('coherenceSpatialAvg.txt', sep='\t', header=4, names=['Date', 'Mean', 'Btemp', 'Bperp', 'Num','NaN1', 'NaN2', 'Nan3']) #Idk why there are 3 columns of NaNs
bad_ifgs = ifg_coh.index[ifg_coh['Mean'] < cohth]
print('# IFGs below coherence threshold: ',len(bad_ifgs), 'out of', len(ifg_coh))

In [ ]:
# Make sure in right directory
os.chdir(ifg_dir)
print('Now in IFG directory', ifg_dir)

# are stackSentinel products geocoded?
ifg_lst = glob.glob('*/filt_fine.unw')

# loop through? what is best way of doing this for 500+ IFGs - I think it's best to just look at the worst overall coherence ones

fig, axs = plt.subplots(len(bad_ifgs), figsize=(30,30))

for i, ele in enumerate(bad_ifgs):
    da = rioxr.open_rasterio(ifg_lst[ele])
    da['band'] = ['amplitude', 'phase']
    ds = da.to_dataset(dim='band')
    #amp = ds['amplitude'] #not plotting amplitude right now
    phs = ds['phase']
    phs = phs.where(phs !=0) #This step takes like 5 seconds per IFG!
    lower_bound = np.nanmean(phs) - 3*np.nanstd(phs)
    upper_bound = np.nanmean(phs) + 3*np.nanstd(phs)
    im = axs[i].imshow(phs, cmap='jet', vmin=lower_bound, vmax=upper_bound)
    fig.colorbar(im)
    axs[i].set_title(ifg_lst[ele].split('/')[0])


In [ ]:
# Exploring ionospheric corrections
ion = rioxr.open_rasterio('/data/eme66/PROCESSDIR/EAR/T28/ion/20221212_20230117/ion_cal/filt.ion')
ion['band'] = ['amplitude', 'phase']
ion_ds = ion.to_dataset(dim='band')
ion_amp = ion_ds['amplitude'] #not plotting amplitude right now
ion_phs = ion_ds['phase']

In [ ]:
print(ion_phs.shape)
print(phs.shape)